In [ ]:
import sys
import os

# Dynamically add SEAL-Python directory to the Python path
# Must follow the steps in README.md to generate Seal-Python directory for library usage
seal_python_path = os.path.join(os.getcwd(), 'SEAL-Python')
sys.path.append(seal_python_path)

from seal import *
import pandas as pd
import numpy as np
import time  # For measuring time

def example_ckks_credit_scores(csv_path):
    # Load the dataset
    data = pd.read_csv(csv_path)

    # Extract columns
    income = data['INCOME'].tolist()
    savings = data['SAVINGS'].tolist()
    debt = data['DEBT'].tolist()

    # Set up encryption parameters for CKKS
    parms = EncryptionParameters(scheme_type.ckks)
    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 60]))
    scale = pow(2.0, 40)
    context = SEALContext(parms)

    # Generate keys
    keygen = KeyGenerator(context)
    secret_key = keygen.secret_key()
    public_key = keygen.create_public_key()

    # Set up encryptor, evaluator, decryptor, and encoder
    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)
    encoder = CKKSEncoder(context)

    # Encode and encrypt data
    income_plain = encoder.encode(income, scale)
    income_encrypted = encryptor.encrypt(income_plain)

    # Measure time for homomorphic sum and average
    start_homomorphic = time.time()

    # Compute sums homomorphically
    income_sum_encrypted = Ciphertext(income_encrypted)
    for i in range(1, len(income)):
        temp_plain = encoder.encode([income[i]], scale)
        temp_encrypted = encryptor.encrypt(temp_plain)
        evaluator.add_inplace(income_sum_encrypted, temp_encrypted)

    # Decrypt and compute average in plaintext
    income_sum_plain = decryptor.decrypt(income_sum_encrypted)
    income_sum_result = encoder.decode(income_sum_plain)
    homomorphic_avg = income_sum_result[0] / len(income)

    end_homomorphic = time.time()

    # Measure time for plaintext sum and average
    start_plaintext = time.time()
    plaintext_avg = sum(income) / len(income)
    end_plaintext = time.time()

    # Print results and timings
    print(f"Homomorphic Average (Income): {homomorphic_avg:.2f}")
    print(f"Plaintext Average (Income): {plaintext_avg:.2f}")

    print(f"Time for Homomorphic Sum and Average: {end_homomorphic - start_homomorphic:.4f} seconds")
    print(f"Time for Plaintext Sum and Average: {end_plaintext - start_plaintext:.4f} seconds")

if __name__ == "__main__":
    # Path to your CSV file
    csv_path = 'credit_score_data.csv'  # Replace with the actual file path
    example_ckks_credit_scores(csv_path)


Homomorphic Average (Income): 121610.02
Plaintext Average (Income): 121610.02
Time for Homomorphic Sum and Average: 3.9918 seconds
Time for Plaintext Sum and Average: 0.0000 seconds
